In [1]:
import pandas as pd 
import numpy as np
import os 
import glob
import re
import time
import random

from Utilities import *

baseball_path = r"C:\Users\james\Documents\MLB\Data2"

In [2]:
# https://mobile-statsv2.sportsnet.ca/web_player_table?league=mlb&season=2023&season_type=reg

In [20]:
batters_steamer = pd.read_csv(os.path.join(baseball_path, "Utilities", "steamer_hitters_weekly_log.csv"), encoding='iso-8859-1')
batters_steamer = batters_steamer[batters_steamer['proj_year'] >= 2022]
batters_steamer.drop_duplicates('steamerid', keep='first')
batters_steamer = batters_steamer[['mlbamid', 'steamerid', 'firstname', 'lastname']]
batters_steamer

,mlbamid,steamerid,firstname,lastname
859140,547172.0,11470,Tony,Wolters
859141,571970.0,13301,Max,Muncy
859142,571875.0,13329,Jake,Lamb
859143,641355.0,15998,Cody,Bellinger
859144,641829.0,16429,Jason,Martin
...,...,...,...,...
999445,670030.0,sa917991,Kole,Enright
999446,657066.0,sa918220,Armond,Upshaw
999447,670622.0,sa919532,Gabriel,Maciel
999448,670582.0,sa920278,Yariel,Gonzalez


In [18]:
batters_steamer[batters_steamer['mlbamid'] == 660670]

,mlbamid,steamerid,firstname,lastname
859408,660670.0,18401,Ronald,Acuna Jr.
862708,660670.0,18401,Ronald,Acuna Jr.
866056,660670.0,18401,Ronald,Acuna Jr.
869459,660670.0,18401,Ronald,Acuna Jr.
872897,660670.0,18401,Ronald,Acuna Jr.
876390,660670.0,18401,Ronald,Acuna Jr.
879936,660670.0,18401,Ronald,Acuna Jr.
883476,660670.0,18401,Ronald,Acuna Jr.
887029,660670.0,18401,Ronald,Acuna Jr.
890624,660670.0,18401,Ronald,Acuna Jr.


In [4]:
pitchers_steamer = pd.read_csv(os.path.join(baseball_path, "Utilities", "steamer_pitchers_weekly_log.csv"), encoding='iso-8859-1')
pitchers_steamer = pitchers_steamer[pitchers_steamer['proj_year'] >= 2022]
pitchers_steamer.drop_duplicates('steamerid', keep='first')
pitchers_steamer = pitchers_steamer[['mlbamid', 'steamerid', 'firstname', 'lastname']]
pitchers_steamer

,mlbamid,steamerid,firstname,lastname
1045975,501985.0,10021,Mike,Minor
1045976,572102.0,10039,Keyvius,Sampson
1045977,554431.0,10058,Tyler,Matzek
1045978,548384.0,10061,Brooks,Raley
1045979,571871.0,10066,Ian,Krol
...,...,...,...,...
1243605,670547.0,sa947763,Miguel,Medrano
1243606,672078.0,sa968102,Eric,Pardinho
1243607,674220.0,sa974592,Adrian,Alcantara
1243608,674372.0,sa975541,Osvaldo,Hernandez


In [5]:
crosswalk = pd.concat([batters_steamer, pitchers_steamer], axis=0).reset_index(drop=True)
crosswalk

,mlbamid,steamerid,firstname,lastname
0,547172.0,11470,Tony,Wolters
1,571970.0,13301,Max,Muncy
2,571875.0,13329,Jake,Lamb
3,641355.0,15998,Cody,Bellinger
4,641829.0,16429,Jason,Martin
...,...,...,...,...
337940,670547.0,sa947763,Miguel,Medrano
337941,672078.0,sa968102,Eric,Pardinho
337942,674220.0,sa974592,Adrian,Alcantara
337943,674372.0,sa975541,Osvaldo,Hernandez


### RotoWire

In [6]:
folder_path = r"C:\Users\james\Documents\MLB\Data2\11. Projections\B. RotoWire"

# Get a list of all file names in the folder
file_names = os.listdir(folder_path)

# Initialize an empty list to store the dataframes
dfs = []

# Loop over the file names and read each CSV file into a dataframe
for file_name in file_names:
    file_path = os.path.join(folder_path, file_name)
    df = pd.read_csv(file_path)
    dfs.append(df)

# Concatenate all dataframes into a single dataframe
roto_df = pd.concat(dfs)

# Keep only the "id" and "player" columns
roto_df = roto_df[["id", "player", "Game Info"]]

# Drop duplicates, keeping the last occurrence
roto_df.drop_duplicates(subset='id', keep="last", inplace=True)

# Reset the index of the final dataframe
roto_df.reset_index(drop=True, inplace=True)

In [7]:
# Split Game Info by team
roto_df[['TeamA', 'TeamB']] = roto_df['Game Info'].str.split("@", expand=True)
roto_df.rename(columns={'id':'rotowire_id'}, inplace=True)
roto_df

,rotowire_id,player,Game Info,TeamA,TeamB
0,13544,Huascar Ynoa,WSN@ATL,WSN,ATL
1,15205,John Means,MIL@BAL,MIL,BAL
2,15396,Casey Mize,DET@KCR,DET,KCR
3,15983,Chris Mariscal,KCR@SEA,KCR,SEA
4,14377,Sean Reynolds,MIA@ATL,MIA,ATL
...,...,...,...,...,...
1870,15162,Mason Martin,PHI@PIT,PHI,PIT
1871,15145,Tres Barrera,CHC@STL,CHC,STL
1872,15136,Ricardo Genoves,BOS@SFG,BOS,SFG
1873,15132,Alexander Campos,OAK@COL,OAK,COL


### Rosters

In [8]:
folder_path = r"C:\Users\james\Documents\MLB\Data2\6. Rosters"

# Initialize an empty list to store the dataframes
dfs = []

# Iterate over the folders in the main directory
for root, dirs, files in os.walk(folder_path):
    for file_name in files:
        if file_name.endswith(".csv"):
            file_path = os.path.join(root, file_name)
            
            try:
                # Read the CSV file as a DataFrame
                df = pd.read_csv(file_path, encoding='iso-8859-1')
                
                # Extract the required columns
                df = df[["fullName", "id", "BBREFTEAM"]]
                
                # Append the DataFrame to the list
                dfs.append(df)
                
            except Exception as e:
                print(f"Error occurred while processing file: {file_path}")
                print(f"Error message: {str(e)}")

# Check if any valid dataframes were obtained
if len(dfs) > 0:
    # Concatenate the dataframes into a single dataframe
    combined_df = pd.concat(dfs, ignore_index=True)
    
    # Print the resulting dataframe
    print(combined_df)
else:
    print("No valid dataframes to concatenate.")
    
combined_df.drop_duplicates(['fullName', 'id', 'BBREFTEAM'], inplace=True, keep='first')


                fullName      id BBREFTEAM
0            Caleb Smith  592761       ARI
1           Carson Kelly  608348       ARI
2       Christian Walker  572233       ARI
3          Cooper Hummel  669450       ARI
4          Corbin Martin  656686       ARI
...                  ...     ...       ...
226982    Patrick Corbin  571578       WSN
226983       Rico Garcia  670329       WSN
226984       Riley Adams  656180       WSN
226985     Stone Garrett  656448       WSN
226986   Trevor Williams  592866       WSN

[226987 rows x 3 columns]


In [9]:
# Merge on player name and team

In [10]:
with_roto = roto_df.merge(combined_df, left_on=['player', 'TeamA'], right_on=['fullName', 'BBREFTEAM'], how='left')
with_roto = with_roto.merge(combined_df, left_on=['player', 'TeamB'], right_on=['fullName', 'BBREFTEAM'], how='left')
with_roto['id_x'].fillna(with_roto['id_y'], inplace=True)
with_roto = with_roto[['player', 'rotowire_id', 'id_x']]
with_roto.rename(columns={'id_x':'mlbamid'}, inplace=True)
print(with_roto)

                player  rotowire_id   mlbamid
0         Huascar Ynoa        13544  660623.0
1           John Means        15205  607644.0
2           Casey Mize        15396  663554.0
3       Chris Mariscal        15983       NaN
4        Sean Reynolds        14377  669308.0
...                ...          ...       ...
1871      Mason Martin        15162  676733.0
1872      Tres Barrera        15145  668663.0
1873   Ricardo Genoves        15136  665648.0
1874  Alexander Campos        15132       NaN
1875    Jacob Gonzalez        14885  669363.0

[1876 rows x 3 columns]


In [11]:
# If there's no match, try just name 

In [12]:
# Iterate over the rows of with_roto dataframe
for index, row in with_roto.iterrows():
    mlbamid = row['mlbamid']
    player = row['player']
    
    # Check if mlbamid is missing
    if pd.isna(mlbamid):
        # Find matching id in combined_df based on player's fullName
        matching_rows = combined_df[combined_df['fullName'] == player]
        
        if len(matching_rows) == 1:
            # Get the corresponding id value
            id_value = matching_rows['id'].values[0]
            
            # Update the with_roto dataframe with the found id
            with_roto.at[index, 'mlbamid'] = id_value
        elif len(matching_rows) > 1:
            unique_ids = matching_rows['id'].unique()
            
            if len(unique_ids) == 1:
                # Get the unique id value
                id_value = unique_ids[0]
                
                # Update the with_roto dataframe with the found id
                with_roto.at[index, 'mlbamid'] = id_value
            else:
                print(f"There is more than one choice for {player}.")
        else:
            print(f"No matching fullName found for {player}.")

# Print the updated with_roto dataframe
print(with_roto)


No matching fullName found for Chris Mariscal.
No matching fullName found for Henri Lartigue.
No matching fullName found for Ian Dawkins.
No matching fullName found for Brock Lundquist.
No matching fullName found for Darell Hernaiz.
No matching fullName found for Carlos Soto.
No matching fullName found for Ronny Brito.
No matching fullName found for Wilson Garcia.
No matching fullName found for Zach Rutherford.
No matching fullName found for Tyler Webb.
No matching fullName found for Tristin English.
No matching fullName found for Michael Smith.
No matching fullName found for Lazaro Armenteros.
No matching fullName found for David Thompson.
No matching fullName found for Andrick Nava.
No matching fullName found for Charcer Burks.
No matching fullName found for Chad Spanberger.
No matching fullName found for Chad McClanahan.
No matching fullName found for Pablo Olivares.
No matching fullName found for Miguel Aparicio.
No matching fullName found for Thomas Dillard.
No matching fullName f

In [ ]:
# Worst-case scenario, we Google it.

In [240]:
for index, row in with_roto.iterrows():
    mlbamid = row['mlbamid']
    player = row['player']
    
    # Check if mlbamid is missing
    if pd.isna(mlbamid):
        try:
            print(player)
            # Call the new_ids function to get the new id
            new_id = new_ids(player, "baseball", "mlb.com")
            print(new_id)
            
            # Update the with_roto dataframe with the new id
            with_roto.at[index, 'mlbamid'] = new_id
        except Exception as e:
            print(f"Error occurred while generating new id for {player}: {str(e)}")
            # return with_roto

with_roto['mlbamid'] = pd.to_numeric(with_roto['mlbamid'])
# Print the updated with_roto dataframe
print(with_roto)


                player  rotowire_id   mlbamid
0         Huascar Ynoa        13544  660623.0
1           John Means        15205  607644.0
2           Casey Mize        15396  663554.0
3       Chris Mariscal        15983  607632.0
4        Sean Reynolds        14377  669308.0
...                ...          ...       ...
1837  Stephen Piscotty        12464  572039.0
1838         Eli White        15737  642201.0
1839    Bubba Thompson        14785  669352.0
1840       Chris Seise        14828  669451.0
1841       Jose Peraza        13190  606299.0

[1842 rows x 3 columns]


In [13]:
crosswalk = pd.merge(crosswalk, with_roto, on='mlbamid', how='left')
crosswalk.drop(columns='player', inplace=True)
crosswalk

,mlbamid,steamerid,firstname,lastname,rotowire_id
0,547172.0,11470,Tony,Wolters,11533.0
1,571970.0,13301,Max,Muncy,13211.0
2,571875.0,13329,Jake,Lamb,12578.0
3,641355.0,15998,Cody,Bellinger,13791.0
4,641829.0,16429,Jason,Martin,NaN
...,...,...,...,...,...
337978,670547.0,sa947763,Miguel,Medrano,NaN
337979,672078.0,sa968102,Eric,Pardinho,NaN
337980,674220.0,sa974592,Adrian,Alcantara,NaN
337981,674372.0,sa975541,Osvaldo,Hernandez,NaN


In [14]:
crosswalk.to_csv(os.path.join(baseball_path, "Utilities", "Crosswalk.csv"), index=False,encoding='iso-8859-1')

In [15]:
crosswalk = pd.read_csv(os.path.join(baseball_path, "Utilities", "Crosswalk.csv"),encoding='iso-8859-1')
crosswalk

,mlbamid,steamerid,firstname,lastname,rotowire_id
0,547172.0,11470,Tony,Wolters,11533.0
1,571970.0,13301,Max,Muncy,13211.0
2,571875.0,13329,Jake,Lamb,12578.0
3,641355.0,15998,Cody,Bellinger,13791.0
4,641829.0,16429,Jason,Martin,NaN
...,...,...,...,...,...
337978,670547.0,sa947763,Miguel,Medrano,NaN
337979,672078.0,sa968102,Eric,Pardinho,NaN
337980,674220.0,sa974592,Adrian,Alcantara,NaN
337981,674372.0,sa975541,Osvaldo,Hernandez,NaN


In [261]:
for index, row in crosswalk.iterrows():
    rotowire_id = row['rotowire_id']
    first = row['firstname']
    last = row['lastname']
    
    # Check if mlbamid is missing
    if pd.isna(rotowire_id):
        try:
            print(first, last)
            # Call the new_ids function to get the new id
            new_id = new_ids(first, last, "mlb.com")
            print(new_id)
            
            # Update the with_roto dataframe with the new id
            crosswalk.at[index, 'rotowire_id'] = new_id
        except Exception as e:
            print(f"Error occurred while generating new id for {player}: {str(e)}")
            break
            # return with_roto

crosswalk['rotowire_id'] = pd.to_numeric(crosswalk['mlbamid'])
# Print the updated with_roto dataframe
print(crosswalk)


         mlbamid steamerid firstname   lastname  rotowire_id
0       547172.0     11470      Tony    Wolters     547172.0
1       571970.0     13301       Max      Muncy     571970.0
2       571875.0     13329      Jake       Lamb     571875.0
3       641355.0     15998      Cody  Bellinger     641355.0
4       641829.0     16429     Jason     Martin     641829.0
...          ...       ...       ...        ...          ...
338053  670547.0  sa947763    Miguel    Medrano     670547.0
338054  672078.0  sa968102      Eric   Pardinho     672078.0
338055  674220.0  sa974592    Adrian  Alcantara     674220.0
338056  674372.0  sa975541   Osvaldo  Hernandez     674372.0
338057  674810.0  sa978399    Victor  Castaneda     674810.0

[338058 rows x 5 columns]


In [262]:
crosswalk.isna().sum()

mlbamid        0
steamerid      0
firstname      0
lastname       0
rotowire_id    0
dtype: int64

To do: read in salaries
Check if mlbamid is in crosswalk
If not, add.
If so, check if steamer id and rotowire id are.
If not, add.

Fill in rotowire_id if missing.

Needs: 
mlbamid
fangraphs_id
draftkings/sportsnet id
rotowire id

Chadwick will have MLB IDs and many fangraphs ID
Missing MLB ID for very recent additions
Missing FanGraphs ID for all rookies

Use all rosters to get a list of unique MLB IDs. Left merge with Chadwick.

Using the approach used above, fill in missing FanGraphs.

Merge with all DK salaries to get draftkings/sportsnet ID

Notes:
We can get an MLBAMID:FanGraphs crosswalk from Steamer. Might need to update from Steamer. 

RotoWire